In [49]:
import json
import csv

## Simplification des fichiers TCL initiaux

In [42]:
# lire fichier json
dataArret = {}
with open('data/sytral_tcl_sytral.tclarret.json') as json_file:
    dataArret = json.load(json_file)

dataLigne = {}
with open('data/sytral_tcl_sytral.tcllignemetrofuni_2_0_0.json') as json_file:
    dataLigne = json.load(json_file)
    
dataLigneTram = {}
with open('data/sytral_tcl_sytral.tcllignetram_2_0_0.json') as json_file:
    dataLigneTram = json.load(json_file)


In [43]:
lignes= set()
for nom_ligne in dataLigne['features']:
    lignes.add((nom_ligne['properties']["code_ligne"],nom_ligne['properties']["ligne"]))
for nom_ligne in dataLigneTram['features']:
    lignes.add((nom_ligne['properties']["code_ligne"],nom_ligne['properties']["ligne"]))

In [44]:
# supprimer les lignes qui ne sont pas dans lignes
dataArretcopy= dataArret.copy()
dataArret['features']=[]
for arret in dataArretcopy['features']:
    add=False
    for transport in arret['properties']['desserte'].split(','):
        for nom_ligne in lignes:
            if transport.split(':')[0] == nom_ligne[0]:
                add=True
                # si nom_ligne n'existe pas, on le crée
                if 'nom_ligne' not in arret['properties']:
                    arret['properties']['nom_ligne']=set()
                arret['properties']['nom_ligne'].add(nom_ligne[1])
    if add:
        arret['properties']['nom_ligne']=list(arret['properties']['nom_ligne'])
        dataArret['features'].append(arret)
                
                

In [45]:
with open('data/sytral_tcl_sytral.tclarret_sans_bus.json', 'w') as outfile:
    json.dump(dataArret, outfile)

## Conversion fichier frequentation en csv

In [109]:
# Lecture du fichier de fréquentation
with open('data/frequentationTCL.txt', encoding='utf-8') as f:
    dataFrequentation = f.readlines()

dataFrequentation = [x.strip() for x in dataFrequentation]


In [117]:
# Parsing du fichier de fréquentation
dict_freq = {}
l_stations = []
for stations in dataFrequentation:
    if stations == "": continue
    else:
        lig = stations.split(':')
        lig[0] = lig[0].strip()
        lig[1] = lig[1].strip()
        if lig[0].replace(' ','')=="Ligne":
            nom_ligne=lig[1]
            dict_freq[nom_ligne]={}
        elif lig[0].replace(' ','')=="Total":
            dict_freq[nom_ligne]["Total"]=lig[1]
        else:
            dict_freq[nom_ligne][lig[0]]=int(lig[1].replace(" ", ""))
            l_stations.append(lig[0])


In [113]:
with open('data/frequentationTCL.csv', 'w', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["ligne","arret","frequentation"])
    for nom_ligne in dict_freq:
        for arret in dict_freq[nom_ligne]:
            if arret!="Total":
                writer.writerow([nom_ligne,arret,dict_freq[nom_ligne][arret]])

In [118]:
dict_freq['D']

{'Bellecour': 67901,
 'Saxe - Gambetta': 36889,
 'Gorge de Loup': 28574,
 'Grange Blanche': 26669,
 'Gare de Vaise': 24282,
 'Guillotière Gabriel Péri': 18004,
 'Sans Souci': 13404,
 'Garibaldi': 13234,
 'Vieux Lyon': 12520,
 'Gare de Vénissieux': 12083,
 'Valmy': 10556,
 'Mermoz - Pinel': 9382,
 'Monplaisir Lumière': 8935,
 'Parilly': 6222,
 'Laennec': 4222,
 'Total': '292 877 voy/j'}

## Ajout des fréquentations dans le fichier des arrêts	

In [119]:
dataArret = {}
with open('data/sytral_tcl_sytral.tclarret_sans_bus.json') as json_file:
    dataArret = json.load(json_file)

# Ajout de la fréquentation dans le fichier json
for arret in dataArret['features']:
    # On initialise la fréquentation.
    arret['properties']['frequentation']={}

    # Un arret peut être desservi par plusieurs lignes
    # On note la frequentation pour chaque ligne grâce à cette boucle
    for nom_ligne in arret['properties']['nom_ligne']:
        # On vérifie que la ligne est dans le fichier de fréquentation
        # car celui date de 2017 (toutes les lignes n'existent pas encore)
        if nom_ligne in dict_freq:
            if arret['properties']['nom'].strip() in dict_freq[nom_ligne]:                
                arret['properties']['frequentation'][nom_ligne]=dict_freq[nom_ligne][arret['properties']['nom'].strip()]
            else:
                print("Erreur : ",nom_ligne,arret['properties']['nom'].strip())                
                arret['properties']['frequentation'][nom_ligne]=0
        else:
            #print("Erreur : ",nom_ligne,arret['properties']['nom'].strip())
            arret['properties']['frequentation'][nom_ligne]=0

In [120]:
# Ecriture du fichier json
with open('data/sytral_tcl_sytral.tclarret_sans_bus.json', 'w') as outfile:
    json.dump(dataArret, outfile)